In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from sklearn import preprocessing
import numpy as np
import sys
import warnings
warnings.filterwarnings("ignore", message="Your PyDataset")
warnings.filterwarnings('ignore', category=UserWarning, module='tensorflow')
warnings.filterwarnings("ignore")
import tensorflow as tf
import os
import logging
# Define o nível de log da Tensorflow para 3 e ignora os demais níveis
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 (INFO), 1 (WARNING), 2 (ERROR), 3 (FATAL)

#####################
# MONTAGEM DO DRIVE #
#####################
from google.colab import drive
if not os.path.ismount('/content/drive'):
    print("Montando o Google Drive...")
    drive.mount('/content/drive')

###########################
# CARREGAMENTO DE MÓDULOS #
###########################

# Caminho dos módulos do projeto armazenados no Google Drive
caminho_modulos = '/content/drive/MyDrive/Colab Notebooks/VC/Projeto_Final/modulos'
from pydoc import importfile
# Importação de módulos Python
modulo_resnet_treino = importfile(caminho_modulos+'/classificadores/resnet_treino.py')
modulo_resnet_teste = importfile(caminho_modulos+'/classificadores/resnet_teste.py')

########################################################################
# CAMINHOS E VARIAVES GLOBAIS
########################################################################

caminho_base = '/content/drive/MyDrive/Colab Notebooks/VC/Projeto_Final'
caminho_todos_datasets = caminho_base + '/datasets/'
selected_dataset = 'coffee_bean'
caminho_imagens_treino = caminho_todos_datasets + selected_dataset + '/train/'
caminho_imagens_val = caminho_todos_datasets + selected_dataset + '/val/'
caminho_imagens_teste = caminho_todos_datasets + selected_dataset + '/test/'
caminho_log = caminho_base + '/log/logger.csv'
caminho_checkpoints = caminho_base + '/checkpoints/cp.weights.h5'
caminho_modelo_resnet_treinado = caminho_base + '/modelos/my_resnet.keras'
caminho_resultados = caminho_base + '/resultados/'
forma_img = (224,224,3)
metrica = 'accuracy'
tam_lote = 32
num_epocas = 20

########################################################################
# FUNÇÕES DE EXECUÇÃO DOS PIPELINES
########################################################################

def pipeline_treinamento():
    print("\n\n")
    ########################################################################
    # OBTEM AS IMAGENS DE TREINAMENTO E VALIDACAO
    ########################################################################

    imagens_treino = modulo_resnet_treino.obter_imagens_treino(
        caminho_imagens_treino,
        forma_img,
        tam_lote)
    imagens_validacao = modulo_resnet_treino.obter_imagens_validacao(
        caminho_imagens_val,
        forma_img,
        tam_lote)
    num_classes = imagens_treino.num_classes

    ########################################################################
    # OBTEM E COMPILA O MODELO
    ########################################################################

    modelo_resnet = modulo_resnet_treino.obter_modelo_resnet(num_classes)
    modelo_compilado = modulo_resnet_treino.compilar_modelo_resnet(modelo_resnet, metrica)

    ########################################################################
    # OBTEM A LISTA DE CALLBACKS
    ########################################################################

    cp = modulo_resnet_treino.obter_checkpoint_callback(caminho_checkpoints)
    log = modulo_resnet_treino.obter_log_callback(caminho_log)
    lr = modulo_resnet_treino.obter_reduce_lr_callback()
    es = modulo_resnet_treino.obter_early_stop_callback()
    lista_callbacks = [cp, log, lr, es]

    ########################################################################
    # TREINA O MODELO
    ########################################################################

    modelo_treinado, historico = modulo_resnet_treino.treinar_modelo(modelo_compilado, imagens_treino, imagens_validacao, num_epocas, caminho_checkpoints,lista_callbacks)

    ########################################################################
    # SALVA MODELO
    ########################################################################

    modulo_resnet_treino.salvar_modelo(modelo_treinado, caminho_modelo_resnet_treinado)

    ########################################################################
    # PLOTA E SALVA HISTÓRICO DO TREINAMENTO
    ########################################################################

    modulo_resnet_treino.plot_historico(historico,metrica,caminho_resultados)


def pipeline_teste():
    print("\n\n")
    ########################################################################
    # OBTEM AS IMAGENS E OS ROTULOS DE TESTE
    ########################################################################

    imagens_teste = modulo_resnet_teste.obter_imagens_teste(
        caminho_imagens_teste,
        forma_img,
        tam_lote)
    rotulos_verdadeiros = imagens_teste.classes
    nomes_das_classes = list(imagens_teste.class_indices.keys())

    ########################################################################
    # CARREGA O MODELO TREINADO A PARTIR DE UM ARQUIVO
    ########################################################################

    modelo_treinado = modulo_resnet_teste.carregar_modelo_resnet(caminho_modelo_resnet_treinado)

    ########################################################################
    # TESTA O MODELO
    ########################################################################

    rotulos_previstos = modulo_resnet_teste.testar_modelo(modelo_treinado, imagens_teste)

    ########################################################################
    # PLOTA E SALVA RESULTADOS DO MODELO
    ########################################################################

    modulo_resnet_teste.matriz_confusao(
        rotulos_verdadeiros,
        rotulos_previstos,
        nomes_das_classes,
        caminho_resultados)

    modulo_resnet_teste.plot_metrica(
        rotulos_verdadeiros,
        rotulos_previstos,
        'precision',
        nomes_das_classes,
        caminho_resultados)

    modulo_resnet_teste.plot_metrica(
        rotulos_verdadeiros,
        rotulos_previstos,
        'recall',
        nomes_das_classes,
        caminho_resultados)

    modulo_resnet_teste.plot_metrica(
        rotulos_verdadeiros,
        rotulos_previstos,
        'f1-score',
        nomes_das_classes,
        caminho_resultados)

########################################################################
# MENU PARA ESCOLHA DOS PIPELINES
 ########################################################################

def menu():
    while True:
        print("\n")
        print("Selecione uma opção:")
        print("1. Executar o pipeline de treinamento")
        print("2. Executar o pipeline de teste")
        print("3. Sair")
        print("\n")

        escolha = input("Digite o número da sua escolha: ")

        if escolha == '1':
            pipeline_treinamento()
        elif escolha == '2':
            pipeline_teste()
        elif escolha == '3':
            print("Execução finalizada!")
            break
        else:
            print("Escolha inválida. Por favor, selecione uma opção válida.")

# Chamada do menu para que o usuário faça a escolha
menu()


################ FIM DO SCRIPT ########################
